# load checkpoint and evaluate

run result on val dataset. no gt.

In [1]:
%matplotlib inline
from utils import *
from model import *
import PIL
import matplotlib
from PIL import Image
from matplotlib import pyplot as plt

SELECT_INTVAL = 10
OUTPUT_MATCH_THREASH = 0.9
OUTPUT_FILE_NAME = os.path.join(WS_PATH, 'output_autumn_night.txt')

EXPRIMENT_NAME = 'experiment4'
TRIAL_NAME = 'Date_2023_05_28_Time_03_22_44_expID_cs27qb9g_trialID_F086c'
CHECKPOINT_NAME = 'bestmodel_epoch_0_sr_0.8791666666666667.pth'


DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
print(DEVICE)


experiment_path = os.path.join(EXPERIMENT_OUTPUT_DIR, EXPRIMENT_NAME)
trail_path = os.path.join(experiment_path, TRIAL_NAME)
checkpoint_path = os.path.join(trail_path, CHECKPOINT_NAME)
print(checkpoint_path)


TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

np.random.seed(0)


/home/rcvlab/anaconda3/envs/py39_VPR/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:1
/home/rcvlab/Workspace/py39_VPR/EEE5346_VPR/EEE5346-VPR/experiment_output/experiment4/Date_2023_05_28_Time_03_22_44_expID_cs27qb9g_trialID_F086c/bestmodel_epoch_0_sr_0.8791666666666667.pth


# load model

In [2]:
model = torch.load(checkpoint_path).to(DEVICE)
model.device = DEVICE
# print(model)

# load test data (copied from run_experiment)

In [3]:
#### imgs txt file to array, shape (n,)
autumn_img_strarr = np.loadtxt(Autumn_val_txt, dtype=str)
night_img_strarr = np.loadtxt(Night_val_txt, dtype=str)
# suncloud_img_strarr = np.loadtxt(Suncloud_val_txt, dtype=str)

#### vo txt file, shape (n,8)
Autumn_vo_strarr = np.loadtxt(Autumn_val_vo, delimiter=',', skiprows=1, dtype=np.double) 
night_vo_strarr = np.loadtxt(Night_val_vo, delimiter=',', skiprows=1, dtype=np.double) 
# suncloud_vo_strarr = np.loadtxt(Suncloud_val_vo, delimiter=',', skiprows=1, dtype=np.double) 


#### load the example loop closure files
# lcd_autumn_night_easy_strarr = np.loadtxt(lcd_autumn_night_easy_path, dtype=str, delimiter=' ')   
# lcd_autumn_night_diff_strarr = np.loadtxt(lcd_autumn_night_diff_path, dtype=str, delimiter=' ')  
# # lcd_autumn_suncloud_easy_strarr = np.loadtxt(lcd_autumn_suncloud_easy_path, dtype=str, delimiter=' ') 
# # lcd_autumn_suncloud_diff_strarr = np.loadtxt(lcd_autumn_suncloud_diff_path, dtype=str, delimiter=' ')  
# lcd_autumn_night_strarr = np.concatenate((lcd_autumn_night_easy_strarr,
#                                           lcd_autumn_night_diff_strarr), axis=0) 
# print(lcd_autumn_night_strarr.shape)
# shape (n,3)
'''final output'''
lcd_autumn_night_strarr = np.loadtxt(lcd_autumn_night_val_path, dtype=str, delimiter=' ')


#### remove the ',' in numpy_str_
for i in range(len(lcd_autumn_night_strarr)):
    lcd_autumn_night_strarr[i][0] = np.char.strip(lcd_autumn_night_strarr[i][0], ',')
    lcd_autumn_night_strarr[i][1] = np.char.strip(lcd_autumn_night_strarr[i][1], ',')
for i in range(len(lcd_autumn_night_strarr)):
    _, lcd_autumn_night_strarr[i][0] = str(lcd_autumn_night_strarr[i][0]).split('/')
    _, lcd_autumn_night_strarr[i][1] = str(lcd_autumn_night_strarr[i][1]).split('/') 
    
#### find sequence for each line and build LCDdata list
autumn_img_idxs, autumn_vo_idxs = \
    imgstr2idx(lcd_autumn_night_strarr[:,0], autumn_img_strarr, Autumn_vo_strarr)
night_img_idxs, night_vo_idxs = \
    imgstr2idx(lcd_autumn_night_strarr[:,1], night_img_strarr, night_vo_strarr)
# autumn_night_gt = [float(g) for g in lcd_autumn_night_strarr[:,2]]

assert len(autumn_img_idxs) == len(night_img_idxs)

#### build the LCDdata list
LCDdata_list = []

for i in range(len(autumn_img_idxs)):
    ## find query
    q1_img_idx, q1_vo_idx = autumn_img_idxs[i], autumn_vo_idxs[i]
    q2_img_idx, q2_vo_idx = night_img_idxs[i], night_vo_idxs[i] 
    q1 = autumn_img_strarr[q1_img_idx]
    q2 = night_img_strarr[q2_img_idx]  
    
    ## find sequence
    q1_left_idx_seq, q1_right_idx_seq, q1_left_relat_dist, q1_right_relat_dist, _, _ = \
        find_seq(autumn_img_strarr, Autumn_vo_strarr, q1_img_idx, q1_vo_idx, DIST_THREASH, SELECT_INTVAL, COUNT_IMG_INTVAL, False)
    q2_left_idx_seq, q2_right_idx_seq, q2_left_relat_dist, q2_right_relat_dist, _, _ = \
        find_seq(night_img_strarr, night_vo_strarr, q2_img_idx, q2_vo_idx, DIST_THREASH, SELECT_INTVAL, COUNT_IMG_INTVAL, False)
 
    ## info for each query
    q1_pos = len(q1_left_idx_seq)
    q1_imgfolder = Autumn_val_imgfolder
    q1_idx_seq = q1_left_idx_seq + [q1_img_idx] + q1_right_idx_seq
    q1_seq = autumn_img_strarr.flatten()[q1_idx_seq]
    q1_relat_dist = q1_left_relat_dist + [0.0] + q1_right_relat_dist
    q1_relat_dist = [(d / DIST_NORM) for d in q1_relat_dist]     
    
    q2_pos = len(q2_left_idx_seq)
    q2_imgfolder = Night_val_imgfolder  
    q2_idx_seq = q2_left_idx_seq + [q2_img_idx] + q2_right_idx_seq
    q2_seq = night_img_strarr.flatten()[q2_idx_seq]
    q2_relat_dist = q2_left_relat_dist + [0.0] + q2_right_relat_dist
    q2_relat_dist = [(d / DIST_NORM) for d in q2_relat_dist]
    
    q1_timestamp, q2_timestamp = imgstr2timestamp(q1_seq), imgstr2timestamp(q2_seq)
    q1_relat_timestamp = [( (t - q1_timestamp[q1_pos]) / TIME_NORM) for t in q1_timestamp]
    q2_relat_timestamp = [( (t - q2_timestamp[q2_pos]) / TIME_NORM) for t in q2_timestamp]  
        
    lcd_data = LCDdata(q1_imgfolder, q2_imgfolder,
                       q1_seq, q2_seq,
                       q1_pos, q2_pos,
                       q1_relat_dist, q2_relat_dist,
                       q1_relat_timestamp, q2_relat_timestamp,
                       0)
    LCDdata_list.append(lcd_data)
    

# evaluate

In [4]:

test_data_list = LCDdata_list

## get outputs
outputs = []
gts = []
model.eval()
for data in test_data_list:
    data.to_torch_seq()
    data.to_device(DEVICE)
    out = model(data) 
    outputs.append(out.item())  
    gts.append(data.gt.item())
    data.release_images()
    

In [6]:
with open(OUTPUT_FILE_NAME, 'w') as f:
    for i,out in enumerate(outputs):
        if out >= OUTPUT_MATCH_THREASH:
            outputs[i] = 1
        elif out < OUTPUT_MATCH_THREASH:
            outputs[i] = 0
        else:
            pass
        f.write(f'{outputs[i]}')
        f.write('\n')